## Here, the Green Infrastructure Designer web site is loaded for users to create custom GI. 

In [ ]:
%%HTML
<iframe id="myiframe" width="1000px" height="900px" src="http://gidesigner.renci.org/GI_Designer_01_31_2017_WithIFrame/GI_HS.html"></iframe>

<script type="text/javascript">

var eventMethod = window.addEventListener;
var eventer = window[eventMethod];
var messageEvent = "message";  

window.addEventListener('message',function(e) {
    var key = e.message ? 'message' : 'data';    
    if(key == "data")
    {
        var data = e[key];    
        var res = data.split("="); 
        var new_key = res[0];
        var new_value = res[1];
        console.log(new_key);
        if(new_key.includes("download_url") )
        {
            IPython.notebook.kernel.execute(new_key + '=\'' + new_value + '\'');
            console.log(new_value);
        }
        else
        {
            var json = JSON.parse(new_value);
            var json_string = JSON.stringify(json);     
            console.log(json_string);               
            IPython.notebook.kernel.execute(new_key + '=\'' + json_string + '\'');        
        }
    };
},false);
</script>

## The steps below assume you have created a new soil and landcover GI design.

In [ ]:
import os
import logging
from utilities import hydroshare
import shutil
from utilities.rhessys import *
%matplotlib inline

w = RHESSysWorkflow(project_name='dead_run_withGI_02_01_17', 
                    gageid='custom',
                    start_date='2008-01-01',
                    end_date='2010-01-01'
                    )

hs = hydroshare.hydroshare()

resource_id = '6dbb0dfb8f3a498881e4de428cb1587c'
# get ETV data bundle for this tutotial stored on HydroShare
content = hs.getResourceFromHydroShare(resource_id)
print('get HydroShare resource Finished')

zipfolder = w.sub_project_folder + '/RHESSys_ETV'
w.create_path(zipfolder)
zipfilepathname = hs.content['DR5_3m_nonburned_DEM_rain_duration_DEM_float_lctest_raingarden.zip']
w.unzip_etv_zip_file_at_path(zipfilepathname, zipfolder)
print('Unzip Finished')

## Step 1 DEM

In [ ]:
os.environ["GDAL_DATA"] = "/opt/conda/share/gdal"
datasource_folder = '/RHESSys_ETV/DR5_3m_nonburned_DEM_rain_duration_DEM_float_lctest_raingarden/'
demfile = w.sub_project_folder + datasource_folder + 'DEM.tif'
#print(demfile)
w.RegisterDEM(w.sub_project_folder, demfile)
print('Finished Step1')

## Step 2 Import Gage

In [ ]:
gagefile = w.sub_project_folder + '/RHESSys_ETV/DR5_3m_nonburned_DEM_rain_duration_DEM_float_lctest_raingarden/gage.shp'
layername = 'gage'
id_attribute = 'gage_id'
id_value = '01589312'

print(gagefile)
w.RegisterGage(w.sub_project_folder, gagefile, layername, id_attribute, id_value)
print('Finished Step2')

## Step 3 Download Soil from GI Design ( NEW - STEP )
### The user will need to have created a soil GI design

In [ ]:
print(soil_download_url);

zipfolder = w.sub_project_folder + '/RHESSys_ETV/GI_Soils'

w.create_path(zipfolder)
zipfilepathname = 'GI_Soil.zip'

!wget $soil_download_url -P $zipfolder

input_location = zipfolder + '/' + zipfilepathname
etv_output_folder_location = zipfolder

w.unzip_etv_zip_file_at_path(input_location, etv_output_folder_location)
print('Finished Step3a')

By default, the GI web services creates a GI soil shapefile called 'MapunitPolyExtended.shp'.<br>
These files need to be copied to the **project directory**

In [ ]:
soil_filename = 'MapunitPolyExtended'
soil_fullpathname_Src = w.sub_project_folder + '/RHESSys_ETV/GI_Soils/'
soil_fullpathname_Dec = w.sub_project_folder
for filename in os.listdir(soil_fullpathname_Src):
    if(filename.startswith(soil_filename)):
        shutil.copy(soil_fullpathname_Src + filename, soil_fullpathname_Dec)
print('Finished Step3b')

The RHESSys workflows automatically adds metadata required by the proceeding functions/tools. If the user has created a new GI Soil data layer, she/he will need to modify the metadata to include soils. <br>
<br>
The **GI_Soil.py** python script does this for you. The **inputs** are: <br>
[1] **-s** MapunitPolyExtended.shp, the name of the soil shapefile. <br>
[2] **-p** The directory to which metadata, intermediate, and final files should be saved.<br>
[3] **--overwrite** Overwrite existing SSURGO features shapefile in project directory.  If not specified, program will halt if a dataset already exists.<br>

In [ ]:
python_file_and_inputs = '/home/jovyan/work/notebooks/GI_Soil.py -s MapunitPolyExtended.shp --overwrite -p ' + w.sub_project_folder
print(python_file_and_inputs)
%run $python_file_and_inputs
print('Finished Step3c')

In [ ]:
w.GenerateSoilPropertyRastersFromSSURGO(w.sub_project_folder)
print('Finished Step3d')

## Step 4 Download Landcover from GI Design ( NEW - STEP )
### The user will need to have created a landcover GI design

In [ ]:
print(landuse_download_url);

zipfolder = w.sub_project_folder + '/RHESSys_ETV/GI_Landuse'

w.create_path(zipfolder)
zipfilepathname = 'GI_Landuse.zip'

!wget $landuse_download_url -P $zipfolder

input_location = zipfolder + '/' + zipfilepathname
etv_output_folder_location = zipfolder

w.unzip_etv_zip_file_at_path(input_location, etv_output_folder_location)
print('Finished Step4a')

In [ ]:
landcover_fullpathname =  w.sub_project_folder + '/RHESSys_ETV/GI_Landuse/GI_LandCover.tif'
options = " -p " + w.sub_project_folder + " -t landcover -r " + landcover_fullpathname + " --force"
print(options)
w.RegisterRasterOptions(options)
print('Finished Step4b')

## Step 5 Prepare LAI Data

In [ ]:
lai_fullpathname_Src = w.sub_project_folder + datasource_folder + 'lai.tif'
lai_fullpathname_Dec = w.sub_project_folder + '/lai_test.tif'
shutil.copy(lai_fullpathname_Src, lai_fullpathname_Dec)

options = " -p " + w.sub_project_folder + " -t lai -r " + lai_fullpathname_Dec + " --force"
#print(options)
w.RegisterRasterOptions(options)
print('Finished Step5')

## Step 6 Create a new GRASS GIS Location

In [ ]:
output = w.CreateGRASSLocationFromDEM(w.sub_project_folder, '"Dead Run 5 near Catonsville, 3m model"')
#print(output)
print('Finished Step6')

## Step 7 Import RHESSys Code

In [ ]:
output = w.ImportRHESSysSource(w.sub_project_folder)
print('Finished Step7')

## Step 8 Import Climate Data

In [ ]:
w.climate_data_fullpathname = w.sub_project_folder + datasource_folder + 'rhessys/clim'
w.ImportClimateData(w.sub_project_folder, w.climate_data_fullpathname)
print('Finished Step8')

## Step 9 Delineate Watershed

In [ ]:
cell_size = 3000
area_estimate = 1.6
output = w.DelineateWatershed(w.sub_project_folder, cell_size, area_estimate)
print('Finished Step9')

## Step 10 GeneratePatchMap

In [ ]:
patch_options = " -p " + w.sub_project_folder + " -t grid"
w.GeneratePatchMapOptions(patch_options)
print('Finished Step10')

## Step 11 Process Soil Texture Map

In [ ]:
output = w.GenerateSoilTextureMap(w.sub_project_folder, options='--overwrite')
print(output)
print('Finished Step11')

## Step 12 Generate derived landcover maps

In [ ]:
leafc_fullpathname_Src = w.sub_project_folder + datasource_folder + 'leafc.tif'
leafc_fullpathname_Dec = w.sub_project_folder + '/leafc_test.tif'
shutil.copy(leafc_fullpathname_Src, leafc_fullpathname_Dec)

options = " -p " + w.sub_project_folder + " -t leafc -r " + leafc_fullpathname_Dec + " --force"
#print(options)
w.RegisterRasterOptions(options)
print('Finished Step12a')

In [ ]:
rootdepth_fullpathname_Src = w.sub_project_folder + datasource_folder + 'rootdepth.tif'
rootdepth_fullpathname_Dec = w.sub_project_folder + '/rootdepth_test.tif'
shutil.copy(rootdepth_fullpathname_Src, rootdepth_fullpathname_Dec)

options = " -p " + w.sub_project_folder + " -t rootdepth -r " + rootdepth_fullpathname_Dec + " --force"
#print(options)
w.RegisterRasterOptions(options)
print('Finished Step12b')

In [ ]:
w.ImportRasterMapIntoGRASS(w.sub_project_folder, 'landcover', 'nearest')
w.ImportRasterMapIntoGRASS(w.sub_project_folder, 'lai', 'nearest')
w.ImportRasterMapIntoGRASS(w.sub_project_folder, 'leafc', 'nearest')
w.ImportRasterMapIntoGRASS(w.sub_project_folder, 'rootdepth', 'nearest')
w.ImportRasterMapIntoGRASS(w.sub_project_folder, 'roof_connectivity', 'nearest')
print('Finished Step12c')

## Step 13 Generate Rules and Reclassify

In [ ]:
options = " -p " + w.sub_project_folder + " -b --includeLaiRules"
w.RegisterLandcoverReclassRulesOptions(options)
print('Finished Step13a')

In [ ]:
#lc_options = " -p " + w.sub_project_folder + " -l "
lc_options = " -p " + w.sub_project_folder + " --makeLaiMap --overwrite"
print(lc_options)
w.GenerateLandcoverMapsOptions(lc_options)
print('Finished Step13b')

## Step 14 Generate World Template

cd ~/data/dead_run_withGI/dead_run_withGI/RHESSys_ETV/DR5_3m_nonburned_DEM_rain_duration_DEM_float_lctest_raingarden/rhessys/de
fs

cp *.def ~/data/dead_run_withGI/dead_run_withGI/rhessys/defs/



In [ ]:
dir_src = w.sub_project_folder + datasource_folder + 'rhessys/defs/'
dir_dst = w.sub_project_folder + '/rhessys/defs/'
for filename in os.listdir(dir_src):
    if filename.endswith('.def'):
        shutil.copy( dir_src + filename, dir_dst)
print('Finished Step14a')

In [ ]:
world_options = " -p " + w.sub_project_folder + " -c dr5_composite5"
##world_options = " -p " + w.sub_project_folder
##rint(world_options)
output = w.GenerateWorldTemplateOptions(world_options)
print(output)
print('Finished Step14b')

## EDITS REQUIRED using vi at terminal window

cd w.sub_project_folder/rhessys/defs <br>
cp landuse_undeveloped.def landuse_test.def <br>
cp stratum_deciduous_BES.def stratum_test.def <br>
Using a text editor modify the template file. i.e. vi landuse_test.def <br>
vi landuse_test.def Change the number of the first line to match the missing ID above. i.e. 0 Save File and Close <br>
vi stratum_test.def Change the number of the first line to match the missing ID above. i.e. 0 Save File and Close <br>
Change your directory to rhessys\templates <br>
cd w.sub_project_folder/rhessys/templates <br>
Using a text editor modify the template file. i.e. vi template <br>
Modify the number of defs/landuse definitions from 3 to 4. <br>
Modify the number of defs/stratum_ definitions from 3 to 4. <br>
Then add the new def file to the template. <br>


## Step 15 Create World File

In [ ]:
output = w.CreateWorldfile(w.sub_project_folder)
#print(output)
print('Finished Step15')

## Step 16 Create Flow Table

In [ ]:
#flow_options = " -p " + w.sub_project_folder + " --routeRoads --routeRoofs -v"
flow_options = " -p " + w.sub_project_folder
print(flow_options)
w.CreateFlowtableOptions(flow_options)
print('Finished Step16')

## Step 17 Initializing vegetation carbon and nitrogen stores 

If the manual edits failed, users will see error messages here. <br>
If errors occur, **Steps 15 to 17** will need to be repeated.


In [ ]:
#flow_options = " -p " + w.sub_project_folder + " --routeRoads --routeRoofs -v"
flow_options = " -p " + w.sub_project_folder
print(flow_options)
w.CreateFlowtableOptions(flow_options)
print('Finished Step17a')

In [ ]:
w.RunLAIRead(w.sub_project_folder)
print('Finished Step17b')

## Step 18 Creating a RHESSys TEC file

In [ ]:
w.RunCmd(w.sub_project_folder, 3)
print('Finished Step18')

## Step 19 Running RHESSys models

In [ ]:
runmodel_options = " -p " + w.sub_project_folder + ' -d \"Test model run\" --basin -pre test -st 2008 1 1 1 -ed 2010 1 1 1 -w world_init -t tec_daily.txt -r world.flow -- -s 0.07041256017 133.552915269 1.81282283058 -sv 4.12459677088 78.3440566535 -gw 0.00736592779294 0.340346799457'
print(runmodel_options)
w.RunModelOptions(runmodel_options)
print('Finished Step19')